In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import csv
import base64
from google.cloud import aiplatform

# Creating a CSV file for the dataset stored in GCS

In [ ]:
!gsutil ls gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass | head

gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3850.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3851.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3852.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3853.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3854.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3855.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3856.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3857.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3858.jpg
gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Glass/R_3859.jpg
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe


In [ ]:
csv_file = "training_dataset.csv"
data_list = []
LABELS = ["Glass","Metal","Organic","Paper","Plastic"]
for label in LABELS:
    data = !gsutil ls gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/{label}
    for i,img in enumerate(data):
        assign = "TRAIN" if i % 5 > 0 else "VALIDATION"
        data_list.append([assign,img,label])

with open(csv_file,"w") as f:
    write = csv.writer(f)
    write.writerows(data_list)

In [ ]:
!gsutil -m cp -r ./training_dataset.csv gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN

Copying file://./training_dataset.csv [Content-Type=text/csv]...
/ [1/1 files][  1.9 MiB/  1.9 MiB] 100% Done                                    
Operation completed over 1 objects/1.9 MiB.                                      


ds = aiplatform.ImageDataset.create(
    display_name="waste_ds",
    gcs_source="gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/training_dataset.csv",
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
    sync=True,
)

# Creating an image classification model using TensorFlow

In [ ]:
from tensorflow import keras
from tensorflow.data import experimental
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
LABELS = ["Glass","Metal","Organic","Paper","Plastic"]

In [ ]:
dataset = tf.data.TextLineDataset("gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/training_dataset.csv").shuffle(20000)
AUTOTUNE = tf.data.experimental.AUTOTUNE

2023-05-22 12:05:38.188765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 12:05:38.202133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 12:05:38.203836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 12:05:38.206949: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
for elem in dataset.take(5):
    print(elem)

tf.Tensor(b'TRAIN,gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Organic/O_1998.jpg,Organic', shape=(), dtype=string)
tf.Tensor(b'TRAIN,gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Organic/O_3016.jpg,Organic', shape=(), dtype=string)
tf.Tensor(b'VALIDATION,gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Metal/R_2064.jpg,Metal', shape=(), dtype=string)
tf.Tensor(b'TRAIN,gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Organic/O_11237.jpg,Organic', shape=(), dtype=string)
tf.Tensor(b'VALIDATION,gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/Organic/O_6510.jpg,Organic', shape=(), dtype=string)


In [ ]:
def parse_csv(data):
    record_defaults = ["split", "path", "label"]
    split, filename, label_string = tf.io.decode_csv(data, record_defaults)
    image_bytes = tf.io.read_file(filename=filename)
    label = tf.math.equal(LABELS, label_string)
    return image_bytes,label

In [ ]:
def decode_image(img,reshape_dims):
    img = tf.image.decode_jpeg(img,channels=3)
    img = tf.image.convert_image_dtype(img,tf.float32)
    return tf.image.resize(img,reshape_dims)

In [ ]:
def data_preprocess(image_bytes,label):
    img = decode_image(image_bytes, [WIDTH, HEIGHT])
    return img, label

In [ ]:
def load_dataset(dataset,batch_size,training=True):
    dataset = dataset.map(parse_csv).cache()
    if training:
        dataset = dataset.map(data_preprocess).repeat(count=None)
    else:
        dataset = dataset.map(data_preprocess).repeat(count=1)
    return dataset.batch(batch_size=batch_size).prefetch(buffer_size=AUTOTUNE)

In [ ]:
#img resolution`
WIDTH = 160
HEIGHT = 160

#MODEL HPARAMS
NUM_FILTERS_1 = 256
NUM_FILTERS_2 = 64
KERNEL_SIZE = 3
POOLING_SIZE = 2
HIDDEN_LAYER_1 = 400
HIDDEN_LAYER_2 = 100

def train_model(dataset):
    
    train_data = load_dataset(dataset,batch_size=64)
    valid_data = load_dataset(dataset,batch_size=32, training=False)
    
    model = Sequential(
        [layers.Conv2D(NUM_FILTERS_1,kernel_size=KERNEL_SIZE,activation="relu",input_shape=(WIDTH,HEIGHT,3)),
        layers.MaxPooling2D(POOLING_SIZE),
        layers.Conv2D(NUM_FILTERS_2,kernel_size=KERNEL_SIZE,activation="relu"),
        layers.MaxPooling2D(POOLING_SIZE),
        layers.Flatten(),
        layers.Dense(HIDDEN_LAYER_1,activation="relu"),
        layers.Dense(HIDDEN_LAYER_2,activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(5),
        layers.Softmax()]
    )
    model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])
    history = model.fit(
        train_data,
        validation_data = valid_data,
        epochs = 50,
        steps_per_epoch=200
    )
    model.save("gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/",save_format='tf')
    RMSE_COLS=["rmse","val_rmse"]
    pd.DataFrame(history.history)[RMSE_COLS].plot()
    
    return model

In [ ]:
model = train_model(dataset)

Epoch 1/50


2023-05-22 12:05:42.653021: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


200/200 [==============================] - 654s 3s/step - loss: 1.1422 - accuracy: 0.5924 - val_loss: 0.9441 - val_accuracy: 0.6539
Epoch 2/50
200/200 [==============================] - 187s 941ms/step - loss: 0.9558 - accuracy: 0.6606 - val_loss: 1.0657 - val_accuracy: 0.6744
Epoch 3/50
200/200 [==============================] - 66s 330ms/step - loss: 0.9357 - accuracy: 0.6580 - val_loss: 0.9126 - val_accuracy: 0.6666
Epoch 4/50
200/200 [==============================] - 66s 332ms/step - loss: 0.8927 - accuracy: 0.6741 - val_loss: 0.8329 - val_accuracy: 0.6854
Epoch 5/50
200/200 [==============================] - 66s 331ms/step - loss: 0.8604 - accuracy: 0.6834 - val_loss: 0.7652 - val_accuracy: 0.7080
Epoch 6/50
200/200 [==============================] - 66s 331ms/step - loss: 0.7985 - accuracy: 0.7034 - val_loss: 0.7039 - val_accuracy: 0.7417
Epoch 7/50
200/200 [==============================] - 66s 331ms/step - loss: 0.7304 - accuracy: 0.7341 - val_loss: 0.6658 - val_accuracy: 0.75

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



200/200 [==============================] - 66s 330ms/step - loss: 0.1111 - accuracy: 0.9666 - val_loss: 0.1162 - val_accuracy: 0.9628
Epoch 19/50
200/200 [==============================] - 66s 329ms/step - loss: 0.0996 - accuracy: 0.9690 - val_loss: 0.1756 - val_accuracy: 0.9461
Epoch 20/50
200/200 [==============================] - 66s 330ms/step - loss: 0.0839 - accuracy: 0.9756 - val_loss: 0.1232 - val_accuracy: 0.9594
Epoch 21/50
200/200 [==============================] - 66s 329ms/step - loss: 0.0627 - accuracy: 0.9804 - val_loss: 0.1100 - val_accuracy: 0.9650
Epoch 22/50
200/200 [==============================] - 66s 329ms/step - loss: 0.0631 - accuracy: 0.9799 - val_loss: 0.1030 - val_accuracy: 0.9707
Epoch 23/50
200/200 [==============================] - 66s 330ms/step - loss: 0.0386 - accuracy: 0.9887 - val_loss: 0.1066 - val_accuracy: 0.9715
Epoch 24/50
200/200 [==============================] - 66s 329ms/step - loss: 0.0558 - accuracy: 0.9841 - val_loss: 0.1312 - val_accurac

2023-05-22 13:12:19.386040: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/assets


KeyError: "None of [Index(['rmse', 'val_rmse'], dtype='object')] are in the [columns]"

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)

NameError: name 'model' is not defined

# Pipeline Ml Ops

In [ ]:
USER_FLAG= "--user"
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.7.0 --upgrade
!pip3 install {USER_FLAG} kfp==1.8.9 google-cloud-pipeline-components==0.2.0

  Using cached google_cloud_aiplatform-1.7.0-py2.py3-none-any.whl (1.6 MB)
  Using cached google_cloud_storage-1.44.0-py2.py3-none-any.whl (106 kB)
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.9.0
    Uninstalling google-cloud-storage-2.9.0:
      Successfully uninstalled google-cloud-storage-2.9.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.25.0
    Uninstalling google-cloud-aiplatform-1.25.0:
      Successfully uninstalled google-cloud-aiplatform-1.25.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pipeline-components 1.0.43 requires google-cloud-aiplatform<2,>=1.14.0, but you have google-cloud-aiplatform 1.7.0 wh

In [ ]:
!pip3 install google-cloud-pipeline-components --upgrade --user

  Using cached google_cloud_pipeline_components-1.0.43-py3-none-any.whl (1.0 MB)
  Using cached google_cloud_storage-2.9.0-py2.py3-none-any.whl (113 kB)
  Using cached google_cloud_aiplatform-1.25.0-py2.py3-none-any.whl (2.6 MB)
INFO: pip is looking at multiple versions of kfp to determine which version is compatible with other requirements. This could take a while.
  Using cached kfp-1.8.22-py3-none-any.whl
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.44.0
    Uninstalling google-cloud-storage-1.44.0:
      Successfully uninstalled google-cloud-storage-1.44.0
  Attempting uninstall: kfp
    Found existing installation: kfp 1.8.9
    Uninstalling kfp-1.8.9:
      Successfully uninstalled kfp-1.8.9
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.7.0
    Uninst

In [ ]:
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component
from kfp.v2.google.client import AIPlatformClient

In [ ]:
PROJECT_ID = 'qwiklabs-asl-02-71a732dc6bb9'
REGION = 'us-central1'
BUCKET_NAME = 'gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL'
PIPELINE_ROOT = "{}/pipeline_root_images/".format(BUCKET_NAME)

In [ ]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [ ]:
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcp_aip

# steps
* dataset
* training
* endpoint
* deployment

In [ ]:
@dsl.pipeline(name="recycle", pipeline_root = PIPELINE_ROOT)
def Recyclepipeline():

    image_data = gcp_aip.ImageDatasetCreateOp(display_name='Waste_Dataset',
                                              gcs_source='gs://qwiklabs-asl-02-71a732dc6bb9/ROSHN-ASL/DATASET/TRAIN/training_dataset.csv',
                                              import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
                                              project = PROJECT_ID
                                             )


    training_out = gcp_aip.AutoMLImageTrainingJobRunOp(project = PROJECT_ID,
                                        display_name='Waste_Training',
                                        prediction_type='classification',
                                        dataset = image_data.outputs['dataset'],
                                        budget_milli_node_hours=8000)



    endpoint_out = gcp_aip.EndpointCreateOp(project = PROJECT_ID,
                             location=REGION,
                             display_name='image_endpoint'
                            )

    gcp_aip.ModelDeployOp(model= training_out.outputs["model"],
                          endpoint = endpoint_out.outputs["endpoint"],
                          automatic_resources_min_replica_count=1,
                          automatic_resources_max_replica_count=1)

In [ ]:
compiler.Compiler().compile(pipeline_func=Recyclepipeline, package_path="waste.json")

In [ ]:
job = aiplatform.PipelineJob(
    display_name="waste-recycle-pipeline",
    template_path="waste.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
    project=PROJECT_ID
)

In [ ]:
job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/153653556327/locations/us-central1/pipelineJobs/recycle-20230522143212
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/153653556327/locations/us-central1/pipelineJobs/recycle-20230522143212')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/recycle-20230522143212?project=153653556327


# CI/CD kubeflow 

In [1]:
# PROJECT_ID = 'qwiklabs-asl-02-71a732dc6bb9'
# REGION = "us-central1"
# ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"

In [2]:
# !gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-02-71a732dc6bb9-kfp-artifact-store/


In [3]:
# !cat kfp-cli_vertex/Dockerfile

cat: kfp-cli_vertex/Dockerfile: No such file or directory


In [ ]:
# KFP_CLI_IMAGE_NAME = "kfp-cli-vertex"
# KFP_CLI_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{KFP_CLI_IMAGE_NAME}:latest"
# KFP_CLI_IMAGE_URI

In [ ]:
# !gcloud builds submit --timeout 15m --tag {KFP_CLI_IMAGE_URI} kfp-cli_vertex

In [ ]:
# SUBSTITUTIONS = f"_REGION={REGION},_PIPELINE_FOLDER=./"
# SUBSTITUTIONS

In [ ]:
# !gcloud builds submit . --config cloudbuild_vertex.yaml --substitutions {SUBSTITUTIONS}